In [ ]:
#############################다음 뉴스 읽어오기
import requests
from bs4 import BeautifulSoup
import pandas as pd
title = []
name = []
urlstr = 'http://media.daum.net/ranking/popular/'
r = requests.get(urlstr)
r.encoding = "utf-8"
bs = BeautifulSoup(r.text, 'html.parser')
titleList = bs.select('.rank_news>ul>li>.cont_thumb>strong>a')
nameList = bs.select(' div.cont_thumb > strong > span')

for titleDom in titleList:
    title.append(titleDom.string)
for nameDom in nameList:
    name.append(nameDom.string)


data = {'newstitle':title, 'newscomname':name}
df = pd.DataFrame(data)
df.set_index('newstitle', inplace=True)
print(df)


df.to_csv("./output/news.csv")

In [ ]:
############################### gs25 읽어오기
from selenium import webdriver
import re
import pandas as pd

driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3) 

driver.get("http://gs25.gsretail.com/gscvs/ko/products/event-goods")
import time
time.sleep(2)

# 2+1 페이지로 이동
links = driver.find_elements_by_css_selector('#TWO_TO_ONE')
for link in links :
    link.click()
    time.sleep(1)

name_list=[]
price_list=[]
PageOldNum = 1

while True:
    # 상품 이름 찾기
    names = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > ul > li > div > p.tit')
    for name in names:
        name_list.append(name.text)
    #print(name_list)


    # 상품 가격 찾기
    price = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > ul > li > div > p.price > span')
    for pri in price:
        content = pri.text
        content = re.sub('원','',content)
        content = re.sub(',','',content)
        price_list.append(content)
    #print(price_list)

    # 다음페이지로 이동
    next_link = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > a.next')
    for link in next_link :
        link.click()
        time.sleep(2)
    
    # 현재 페이지 넘버를 읽고 이전페이지 넘버와 비교 (마지막 페이지 확인용)-----이전 페이지 초기값은 반복문 시작전에 1로 지정
    curr_Page = driver.find_elements_by_css_selector('#contents > div.cnt > div.cnt_section.mt50 > div > div > div:nth-child(5) > div > span > a.on')
    for page in curr_Page:
        currpage = page.text
    
    PageNewNum = int(currpage)
    if PageNewNum == PageOldNum:
        break
        
    # 이전페이지를 현재페이지로 변경
    PageOldNum = PageNewNum

    

########## 결과 ###########
print(len(name_list)) #742

print(len(price_list)) #742



data = {'goodsname':name_list, 'goodsprice':price_list}
df = pd.DataFrame(data)
df.set_index('goodsname', inplace=True)
print(df)


df.to_csv("./output/gs25_twotoone.csv")



######### CSV파일로 저장 #########
# with open('C:/Temp/gs25_twotoone.csv', "wt", encoding="utf-8") as f:
#     f.write('goodsname,goodsprice\n')  
#     for i in range(len(name_list)):
#         f.write(name_list[i]+","+price_list[i]+'\n')  
        
######### CSV파일로 저장 #########
# import csv

# with open('C:/Temp/gs25_twotoone.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['goodsname','goodsprice'])
#     for i in range(len(name_list)):
#         writer.writerow((name_list[i],price_list[i]))